This notebook is associated with the paper "The relative class number one problem for function fields, I" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5) and also depends on Magma (tested using version
2.26-9).

In this notebook, we run the exhaustion over Weil polynomials for $q=3$ and $q=4$, and compute all cyclic covers with relative class number 1. Equations for curves with a given Weil polynomial are taken from LMFDB.

In [244]:
load("../Shared/weil_poly_utils.sage")
load("../Shared/cyclic_covers.sage")
from collections import defaultdict
import re

Collect candidate Weil polynomials for $C$ based on $q,d,g,g'$.

In [222]:
P.<T> = QQ[]

In [223]:
candidates = {t: [] for t in [(3,2,2,3), (3,2,2,4), (3,2,3,5), \
                              (3,2,3,6), (3,2,4,7), (3,3,2,4), \
                              (4,2,2,3), (4,2,2,4), (4,2,3,5), (4,3,2,4)]}

In [224]:
for q in [3,4]:
    for g in [2,3,4]:
        tmp = [s for s in candidates if s[0] == q and s[2] == g]
        if not tmp:
            continue
        l = P.weil_polynomials(q=q,d=2*g)
        # Check resultant criterion.
        l = [u for u in l if not _nojac_serre(u, q=q)]
        # Check positivity for q^3-points.
        l = [u for u in l if check_curve_positivity(u, 3, q=q)]
        print('Number of eligible Weil polynomials for (q,g) = {}: {}'.format((q,g), len(l)))
        for s in tmp:
            d = s[1]
            g1 = s[3]
            delta = g1 - g - (d-1)*(g-1)
            if delta == 0:
                t = 0
            elif delta == 1 and d == 2 and q == 3:
                t = 2
            elif delta == 1 and d == 2 and q == 4:
                t = 1
            else:
                raise ValueError
            for u in l:
                tmp2 = point_count_from_weil_poly(u, 3, q=q)
                # Check (5.3) for i=1.
                if not (tmp2[0] >= q*(g1-g)):
                    continue
                # Check (7.6) for i=1.
                if d == 2 and not (tmp2[1] >= 2*tmp2[0] + q*(q-2)*(g1-g) - t):
                    continue
                # Check (7.5).
                if delta == 0 and not (u(1)%d == 0):
                    continue
                # Check (7.2).
                if q == 4 and d == 2 and not all(u[i]%2 == 0 for i in range(2, g+1)):
                    continue
                if q == 3 and d == 3 and not all(u[i]%3 == 0 for i in range(2, g+1)):
                    continue
                candidates[s].append(label_from_weil_poly(u))
            print("Candidates for t = {}: {}".format(s, candidates[s]))

Number of eligible Weil polynomials for (q,g) = (3, 2): 52
Candidates for t = (3, 2, 2, 3): ['2.3.c_g', '2.3.b_e', '2.3.b_c', '2.3.a_g', '2.3.a_e', '2.3.a_c', '2.3.ab_e', '2.3.ab_c', '2.3.ab_a']
Candidates for t = (3, 2, 2, 4): ['2.3.c_h', '2.3.c_g', '2.3.c_f']
Candidates for t = (3, 3, 2, 4): ['2.3.c_g', '2.3.c_d']
Number of eligible Weil polynomials for (q,g) = (3, 3): 556
Candidates for t = (3, 2, 3, 5): ['3.3.c_j_m', '3.3.c_i_k', '3.3.c_h_m', '3.3.c_h_k', '3.3.c_h_i', '3.3.c_g_m', '3.3.c_g_k', '3.3.c_g_i']
Candidates for t = (3, 2, 3, 6): []
Number of eligible Weil polynomials for (q,g) = (3, 4): 9315
Candidates for t = (3, 2, 4, 7): []
Number of eligible Weil polynomials for (q,g) = (4, 2): 87
Candidates for t = (4, 2, 2, 3): ['2.4.d_i', '2.4.b_g', '2.4.b_e', '2.4.b_c', '2.4.ab_g', '2.4.ab_e', '2.4.ab_c', '2.4.ab_a']
Candidates for t = (4, 2, 2, 4): []
Candidates for t = (4, 3, 2, 4): ['2.4.g_q', '2.4.f_m', '2.4.e_l', '2.4.e_i', '2.4.d_h', '2.4.d_e']
Number of eligible Weil polyno

Tabulate curves with a given Weil polynomial from LMFDB.

In [225]:
curves_by_poly = {}

In [234]:
Q0.<w> = PolynomialRing(GF(3))
curves_by_poly[3,2] = {
    '2.3.ab_a': [2*w^6+w^4+2*w],
    '2.3.ab_c': [w^6+w^5+2*w^3+w^2+w+2, 2*w^5+w^2+2*w],
    '2.3.ab_e': [2*w^6+2*w^4+w^3+2*w^2+w],
    '2.3.a_c': [w^5+w, w^6+w^5+2*w^3+w+2],
    '2.3.a_e': [w^6+2*w^5+w+2],
    '2.3.a_g': [],
    '2.3.b_c': [2*w^6+2*w^5+w^3+2*w^2+2*w+1, w^5+2*w^2+w],
    '2.3.b_e': [w^6+w^4+2*w^3+w^2+2*w],
    '2.3.c_d': [w^6+w^4+2*w^3+2*w+1, w^6+w^5+2*w^4+2*w^2+w+1],
    '2.3.c_f': [w^6+2*w^5+w^3+w^2+2*w+1],
    '2.3.c_g': [w^5+2*w^4+w^3+2*w^2+w, w^6+w^5+2*w^4+2*w^3+2*w^2+w+1],
    '2.3.c_h': [w^6+2*w^4+w^3+2*w^2+1]
}

In [227]:
Q1.<x,y,z> = PolynomialRing(GF(3))
curves_by_poly[3,3] = {
    '3.3.c_g_i': [[2*w^8+2*w^7+w^5+w^3+w^2+2*w+1, 
                   w^8+2*w^7+2*w^5+w^4+2*w^3+2*w+1,
                   w^8+2*w^7+2*w^5+2*w^3+w^2+w,
                   w^8+w^7+w^4+2*w^3+w+1, 
                   w^7+2*w^6+w^5+2*w^3+2*w^2+2*w],
                  [2*x^3*z+x^2*y^2+x*y*z^2+x*z^3+y^4, 
                   x^3*y+x^3*z+2*x^2*y^2+2*x^2*y*z+x*y^2*z+x*y*z^2+x*z^3+y^3*z,
                   x^3*y+2*x^3*z+2*x^2*y^2+2*x*y^2*z+x*y*z^2+x*z^3+y^3*z, 
                   x^4+x^3*z+x^2*y^2+x*y^3+x*z^3+y^2*z^2,
                   x^3*y+x^3*z+x^2*y^2+x^2*y*z+x^2*z^2+x*y^3+x*z^3+y^2*z^2]],
    '3.3.c_g_k': [[], 
                  [x^4+2*x^2*y^2+x*y^2*z+x*y*z^2+x*z^3+y^3*z]],
    '3.3.c_g_m': [[2*w^7+w^5+2*w^4+2*w^3+2*w^2+1,
                   w^8+2*w^7+w^4+2*w],
                  [x^3*y+x^3*z+2*x^2*z^2+x*y*z^2+x*z^3+y^4,
                   x^3*y+x^2*y^2+x^2*z^2+2*x*y^2*z+x*z^3+y^3*z]],
    '3.3.c_h_i': [[w^8+w^4+2],
                  [2*x^4+2*x^3*y+x^3*z+2*x^2*y^2+x^2*y*z+x*y^2*z+x*y*z^2+x*z^3+y^3*z,
                   x^4+2*x^3*z+2*x^2*y^2+2*x^2*y*z+2*x*y^2*z+x*y*z^2+x*z^3+y^3*z,
                   x^3*y+x^3*z+2*x^2*y^2+2*x^2*z^2+x*y^3+x*z^3+y^2*z^2]],
    '3.3.c_h_k': [[w^8+2*w^7+w^6+w^5+2*w^4+2*w^3+w^2+w+2], 
                  []],
    '3.3.c_h_m': [[w^8+w^6+2, 
                   w^8+2*w^7+2*w^6+w^5+w^4+w^3+w^2+w+1], 
                  [x^4+x^3*z+x^2*z^2+x*y^2*z+x*z^3+y^4]],
    '3.3.c_i_k': [[], 
                  [2*x^4+x^3*z+2*x^2*z^2+2*x*y^2*z+x*z^3+y^4,
                   x^4+2*x^3*y+x^3*z+x*y^2*z+x*y*z^2+x*z^3+y^3*z]],
    '3.3.c_j_m': [[], [2*x^4+x^3*y+x^3*z+x*y^3+x*z^3+y^2*z^2]]
}

In [228]:
curves_by_poly[3,4] = {
    '4.3.f_v_ca_eg': [[], []]
}

In [240]:
K.<a> = GF(4)
Q0.<w> = PolynomialRing(K)
curves_by_poly[4,2] = {
    '2.4.ab_a': [(w, w^5+a*w^3+w)],
    '2.4.ab_c': [(w, a*w^5+a*w^2+w)],
    '2.4.ab_e': [(w, w^5+w)],
    '2.4.ab_g': [(w, a*w^5+a*w^3+a*w^2+w)],
    '2.4.b_ab': [(w^3+a+1, a*w^6+(a+1)*w^4+a*w^3+a*w+a)],
    '2.4.b_c': [(w, a*w^5+w), (w, (a+1)*w^5+w)],
    '2.4.b_e': [(w, w^5+a*w^2+w)],
    '2.4.b_f': [(w^3+w+1, w^3+w+1), (w^3+a, a*w^4+(a+1)*w)],
    '2.4.b_g': [(w, a*w^5+a*w^3+w), (w, (a+1)*w^5+(a+1)*w^3+w)],
    '2.4.c_d': [(w^2+w+a, a*w^5+(a+1)*w^4+w^3+(a+1)*w^2+(a+1)*w+a+1),
                (w^2+w+a+1, (a+1)*w^5+(a+1)*w^4+w^3+(a+1)*w^2+a*w),
                (w^2+w+a, a*w^5+a+1)],
    '2.4.d_e': [],
    '2.4.d_h': [(w^3+w+1, a*w^5+a*w^4+a*w^3+a*w), (w^3+w+1, a*w^5+a*w^4+(a+1)*w^3+(a+1)*w+1)],
    '2.4.d_i': [(w, w^5+w^3+w)],
    '2.4.e_i': [(Q0(1), w^5+w^4)],
    '2.4.e_l': [(w^2+w+a+1, w^5+w^3+(a+1)*w^2+w+a), (w^2+w+a+1, w^5+(a+1)*w^4+w^3+w^2)],
    '2.4.f_m': [],
    '2.4.g_q': [],
}

In [230]:
Q1.<x,y,z> = PolynomialRing(K)
curves_by_poly[4,3] = {
    '3.4.d_m_u': [[], [a*x^4+a*x^3*y+a*x^3*z+a*x^2*y^2+a*x^2*y*z+x^2*z^2+x*y*z^2+x*z^3+y^3*z,
                  a^2*x^4+a^2*x^3*y+a^2*x^3*z+a^2*x^2*y^2+a^2*x^2*y*z+x^2*z^2+x*y*z^2+x*z^3+y^3*z,
                  a^2*x^4+x^3*y+x^3*z+a*x^2*y*z+x*y^3+x*y^2*z+x*y*z^2+x*z^3+y^2*z^2,
                  a*x^4+x^3*y+x^3*z+a^2*x^2*y*z+x*y^3+x*y^2*z+x*y*z^2+x*z^3+y^2*z^2]],
    '3.4.d_m_w': [[], []],
    '3.4.d_m_y': [[], [a^2*x^4+a*x^3*y+a*x^3*z+x^2*y*z+x*y^3+x*y^2*z+x*y*z^2+x*z^3+y^2*z^2,
                  a*x^4+a^2*x^3*y+a^2*x^3*z+x^2*y*z+x*y^3+x*y^2*z+x*y*z^2+x*z^3+y^2*z^2]],    
}

Convert representations of curves to Magma function fields.

In [231]:
def function_fields_by_weil_poly(s):
    l = s.split(".")
    g = Integer(l[0])
    q = Integer(l[1])
    ans = []
    # Hyperelliptics.
    m = curves_by_poly[q,g][s] if g == 2 else curves_by_poly[q,g][s][0]
    for u in m:
        if q == 3:
            u = [u]
        yield magma.HyperellipticCurve(*reversed(u)).FunctionField().AlgorithmicFunctionField()
    # Plane quartics for g=3.
    if g == 3:
        for u in curves_by_poly[q,g][s][1]:
            proj = magma.ProjectiveSpace(u.parent())
            yield proj.Scheme([u]).FunctionField().AlgorithmicFunctionField()
    # Genus 4 curves.
    if g == 4:
        assert not curves_by_poly[q,g][s][1]

Check the transcription by verifying that the Weil polynomials are correctly assigned.

In [232]:
for (q,g) in curves_by_poly:
    for s in curves_by_poly[q,g]:
        for F in function_fields_by_weil_poly(s):
            u = F.ZetaFunction().Numerator().sage()(T).reverse()
            assert label_from_weil_poly(u) == s

Exhaust over unramified abelian extensions of a Magma function field of a fixed prime degree to find instances of a particular relative Weil polynomial.

In [265]:
covers_by_poly = defaultdict(dict)
for (q,d,g,g1) in candidates:
    print(q,d,g,g1)
    for s in candidates[q,d,g,g1]:
        for F in function_fields_by_weil_poly(s):
            delta = g1-g-(d-1)*(g-1)
            if q == 3 and d == 2:
                delta *= 2
            tmp = match_weil_poly(F, d, delta=delta, q=q)
            if tmp:
                if s not in covers_by_poly[q,g,g,g1]:
                    covers_by_poly[q,d,g,g1][s] = []
                covers_by_poly[q,d,g,g1][s] += tmp

3 2 2 3
3 2 2 4
3 2 3 5
3 2 3 6
3 2 4 7
3 3 2 4
4 2 2 3
4 2 2 4
4 2 3 5
4 3 2 4


Generate a LaTeX-formatted table of results.

In [270]:
print(r'\begin{tabular}{c|c|c|c|c|l}')
print(r"$q_F$ & $d$ & $g_F$ & $g_{F'}$ & $J(C)$ & $F$ \\")
for (q,d,g,g1) in sorted(candidates.keys()):
    if not covers_by_poly[q,d,g,g1]:
        continue
    if (q,d,g,g1) == (4,2,2,3):
        print(r'\hline')
    print(r'\hline')
    for s in sorted(covers_by_poly[q,d,g,g1]):
        s1 = r"\avlink{{{}}}".format(s)
        for (F, _) in covers_by_poly[q,d,g,g1][s]:
            F1 = F.RationalExtensionRepresentation()
            u = F1.DefiningPolynomial().sage()
            P0 = PolynomialRing(u.base_ring().base_ring(), names='x')
            u1 = u.change_ring(P0)
            u2 = PolynomialRing(u1.base_ring(), names='y')(u1)
            print(r'${}$ & ${}$ & ${}$ & ${}$ & ${}$ & ${}$ \\'.format(q, d, g, g1, 
                                                        re.sub('_', r'\_', s1),
                                                        latex(u2)))
print(r'\end{tabular}')

\begin{tabular}{c|c|c|c|c|l}
$q_F$ & $d$ & $g_F$ & $g_{F'}$ & $J(C)$ & $F$ \\
\hline
$3$ & $2$ & $2$ & $3$ & $\avlink{2.3.ab\_c}$ & $y^{2} + x^{5} + 2 x^{2} + x$ \\
$3$ & $2$ & $2$ & $3$ & $\avlink{2.3.ab\_e}$ & $y^{2} + x^{6} + x^{4} + 2 x^{3} + x^{2} + 2 x$ \\
$3$ & $2$ & $2$ & $3$ & $\avlink{2.3.b\_c}$ & $y^{2} + 2 x^{5} + x^{2} + 2 x$ \\
$3$ & $2$ & $2$ & $3$ & $\avlink{2.3.b\_e}$ & $y^{2} + 2 x^{6} + 2 x^{4} + x^{3} + 2 x^{2} + x$ \\
\hline
$3$ & $2$ & $2$ & $4$ & $\avlink{2.3.c\_h}$ & $y^{2} + 2 x^{6} + x^{4} + 2 x^{3} + x^{2} + 2$ \\
\hline
$3$ & $2$ & $3$ & $5$ & $\avlink{3.3.c\_g\_i}$ & $y^{2} + 2 x^{8} + x^{7} + x^{5} + x^{3} + 2 x^{2} + 2 x$ \\
$3$ & $2$ & $3$ & $5$ & $\avlink{3.3.c\_g\_m}$ & $y^{2} + x^{7} + 2 x^{5} + x^{4} + x^{3} + x^{2} + 2$ \\
\hline
$3$ & $3$ & $2$ & $4$ & $\avlink{2.3.c\_d}$ & $y^{2} + 2 x^{6} + 2 x^{4} + x^{3} + x + 2$ \\
$3$ & $3$ & $2$ & $4$ & $\avlink{2.3.c\_g}$ & $y^{2} + 2 x^{6} + 2 x^{5} + x^{4} + x^{3} + x^{2} + 2 x + 2$ \\
\hline
\hline
$4$ &